In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import threading

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
MUSINSA_SHOES_LIST_URL = "https://www.musinsa.com/categories/item/002008"

In [4]:
def get_chrome_driver():
    # 1. 브라우저 옵션 세팅
    chrome_options = webdriver.ChromeOptions()
    
    # 2. driver 생성
    driver = webdriver.Chrome(
        service = Service(ChromeDriverManager().install()),
        options = chrome_options
    )
    return driver

In [5]:
def crawl_detail(link):
    rule = re.compile("[^\t\n\f\" \"]+")
    headers = {'User-agent' : UserAgent().chrome}
    response = requests.get(link, headers = headers)
    page = response.content
    soup = BS(page, "html.parser")
    product_article = soup.select_one(".product_article")
    try:
        시즌 = " ".join(re.findall(rule, product_article.select(".product_article_contents strong")[1].text))
    except:
        시즌 = None
    try:
        성별 = product_article.select_one(".txt_gender").text.replace(" ","").replace("\n","")
    except:
        성별 = None
    try:
        별점 = product_article.select_one(".prd-score__rating").text
    except:
        별점 = None
    이미지 = ""
    product_thumb = soup.select_one(".product_thumb")
    try:
        product_img_list = product_thumb.select("img")
        for item_img in range(len(product_img_list)):
            이미지 += "https:" + product_img_list[item_img]["src"]
            if item_img == len(product_img_list) - 1:
                break
            이미지 += ", "
    except:
        pass
    
    return (시즌, 성별, 별점, 이미지)

In [6]:
# def crawl_detail(link, item, 브랜드, 상품명, 가격, 별점개수, 좋아요개수):
#     rule = re.compile("[^\t\n\f\" \"]+")
#     headers = {'User-agent' : UserAgent().chrome}
#     response = requests.get(link, headers = headers)
#     page = response.content
#     soup = BS(page, "html.parser")
#     product_article = soup.select_one(".product_article")
#     try:
#         시즌 = " ".join(re.findall(rule, product_article.select(".product_article_contents strong")[1].text))
#     except:
#         시즌 = None
#     try:
#         성별 = product_article.select_one(".txt_gender").text.strip()
#     except:
#         성별 = None
#     try:
#         별점 = product_article.select_one(".prd-score__rating").text
#     except:
#         별점 = None
#     이미지 = []
#     product_thumb = soup.select_one(".product_thumb")
#     product_img_list = product_thumb.select("img")
#     for item_img in range(len(product_img_list)):
#         이미지.append("https:" + product_img_list[item_img]["src"])
    
#     product_dict_list[item]["브랜드"] = 브랜드
#     product_dict_list[item]["상품명"] = 상품명
#     product_dict_list[item]["가격"] = 가격
#     product_dict_list[item]["별점개수"] = 별점개수
#     product_dict_list[item]["좋아요개수"] = 좋아요개수
#     product_dict_list[item]["시즌"] = 시즌
#     product_dict_list[item]["성별"] = 성별
#     product_dict_list[item]["별점"] = 별점
#     product_dict_list[item]["이미지"] = 이미지

In [7]:
def crawl_list(link):
    product_dict_list = []
    rule = re.compile("[0-9]+")
    driver = get_chrome_driver()
    driver.get(MUSINSA_SHOES_LIST_URL)
    page = driver.page_source
    soup = BS(page, "html.parser")
    product_lst = soup.select("#searchList > .li_box")
    driver.quit()
    for item in range(len(product_lst)):
        print(item, "번째")
        product_dict_list.append({})
        브랜드 = product_lst[item].select_one(".item_title").text
        상품명 = product_lst[item].select_one(".list_info > a")["title"]
        가격 = int(re.findall(rule, product_lst[item].select_one(".price").text.replace(",",""))[-1])
        try:
            별점개수 = int(product_lst[item].select_one(".count").text.replace(",",""))
        except:
            별점개수 = None
        try:
            좋아요개수 = int(product_lst[item].select_one(".txt_cnt_like").text.replace(",",""))
        except:
            좋아요개수 = None
        상세링크 = "https:" + product_lst[item].select_one(".list_info > a")["href"]
        
        시즌, 성별, 별점, 이미지 = crawl_detail(상세링크)
        product_dict_list[item]["브랜드"] = 브랜드
        product_dict_list[item]["상품명"] = 상품명
        product_dict_list[item]["가격"] = 가격
        product_dict_list[item]["별점개수"] = 별점개수
        product_dict_list[item]["좋아요개수"] = 좋아요개수
        product_dict_list[item]["시즌"] = 시즌
        product_dict_list[item]["성별"] = 성별
        product_dict_list[item]["별점"] = 별점
        product_dict_list[item]["이미지"] = 이미지
        
    return product_dict_list

In [8]:
# def crawl_list(link):
#     rule = re.compile("[0-9]+")
#     driver = get_chrome_driver()
#     driver.get(MUSINSA_SHOES_LIST_URL)
#     page = driver.page_source
#     soup = BS(page, "html.parser")
#     product_lst = soup.select("#searchList > .li_box")
#     driver.quit()
#     for item in range(len(product_lst)):
#         print(item, "번째")
#         product_dict_list.append({})
#         브랜드 = product_lst[item].select_one(".item_title").text
#         상품명 = product_lst[item].select_one(".list_info > a")["title"]
#         가격 = int(re.findall(rule, product_lst[item].select_one(".price").text.replace(",",""))[-1])
#         try:
#             별점개수 = int(product_lst[item].select_one(".count").text.replace(",",""))
#         except:
#             별점개수 = None
#         try:
#             좋아요개수 = int(product_lst[item].select_one(".txt_cnt_like").text.replace(",",""))
#         except:
#             좋아요개수 = None
#         상세링크 = "https:" + product_lst[item].select_one(".list_info > a")["href"]
        
#         th = threading.Thread(target = crawl_detail, args=(상세링크, item, 브랜드, 상품명, 가격, 별점개수, 좋아요개수))
#         th.start()
        
#     return product_dict_list

In [9]:
data = crawl_list(MUSINSA_SHOES_LIST_URL)

0 번째
1 번째
2 번째
3 번째
4 번째
5 번째
6 번째
7 번째
8 번째
9 번째
10 번째
11 번째
12 번째
13 번째
14 번째
15 번째
16 번째
17 번째
18 번째
19 번째
20 번째
21 번째
22 번째
23 번째
24 번째
25 번째
26 번째
27 번째
28 번째
29 번째
30 번째
31 번째
32 번째
33 번째
34 번째
35 번째
36 번째
37 번째
38 번째
39 번째
40 번째
41 번째
42 번째
43 번째
44 번째
45 번째
46 번째
47 번째
48 번째
49 번째
50 번째
51 번째
52 번째
53 번째
54 번째
55 번째
56 번째
57 번째
58 번째
59 번째
60 번째
61 번째
62 번째
63 번째
64 번째
65 번째
66 번째
67 번째
68 번째
69 번째
70 번째
71 번째
72 번째
73 번째
74 번째
75 번째
76 번째
77 번째
78 번째
79 번째
80 번째
81 번째
82 번째
83 번째
84 번째
85 번째
86 번째
87 번째
88 번째
89 번째


In [10]:
datas = pd.DataFrame(data)
datas.head(15)

,브랜드,상품명,가격,별점개수,좋아요개수,시즌,성별,별점,이미지
0,드로우핏,오버사이즈 울 트렌치 코트 [BEIGE],156700,7355.0,15866.0,2023 S/S,남,4.8,https://image.msscdn.net/images/goods_img/2018...
1,드로우핏,오버사이즈 울 트렌치 코트 [BLACK],156700,4170.0,11668.0,2023 S/S,남,4.8,https://image.msscdn.net/images/goods_img/2018...
2,드로우핏,울 트렌치 싱글 발마칸 코트 [GREYISH BEIGE],167200,117.0,1568.0,2023 S/S,남,4.9,https://image.msscdn.net/images/goods_img/2023...
3,굿라이프웍스,오버사이즈 트렌치코트 블랙,119800,344.0,3810.0,2021 F/W,남,4.8,https://image.msscdn.net/images/goods_img/2021...
4,플레어업,오버사이즈 트렌치코트 (FL-012),156700,2573.0,8048.0,2023 ALL,남여,4.8,https://image.msscdn.net/images/goods_img/2017...
5,인사일런스,오버사이즈 트렌치 코트 BLACK,179000,1253.0,5621.0,2023 S/S,남,4.8,https://image.msscdn.net/images/goods_img/2020...
6,드로우핏,울 블렌딩 발마칸 오버 코트 [BLACK],167200,544.0,4843.0,2023 S/S,남,4.9,https://image.msscdn.net/images/goods_img/2021...
7,드로우핏,울 트렌치 싱글 발마칸 코트 [BLACK],167200,43.0,643.0,2023 S/S,남,4.9,https://image.msscdn.net/images/goods_img/2023...
8,드로우핏,오버사이즈 울 트렌치 코트 [GREYISH BEIGE],156700,1021.0,3507.0,2023 S/S,남,4.9,https://image.msscdn.net/images/goods_img/2021...
9,닉앤니콜,[21FW] 오버사이즈 더블 트렌치 코트_그레이,99500,613.0,7141.0,2021 F/W,여,4.6,https://image.msscdn.net/images/goods_img/2021...


In [11]:
datas.to_csv("musinsa.csv", encoding="cp949")

In [12]:
import pymysql

In [18]:
conn = pymysql.connect(
    host = "127.0.0.1",
    port = 3306,
    user = "root",
    password = "0109",
    autocommit = True,
    charset = "utf8mb4"
)
cur = conn.cursor()
cur.execute("drop database if exists musinsa;")
cur.execute("create database if not exists musinsa;")
cur.execute("use musinsa")
cur.execute("create table `musinsa` (\
            `idx` int not null auto_increment,\
            `브랜드` varchar(20) default null,\
            `상품명` varchar(50) not null,\
            `가격` int default null,\
            `별점개수` int default null,\
            `좋아요개수` int default null,\
            `시즌` varchar(20) default null,\
            `성별` varchar(20) default null,\
            `별점` float default null,\
            `이미지` text default null,\
            primary key (`idx`)\
            ) ENGINE=InnoDB CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;")

0

In [20]:
for i in range(len(datas)):
    a1 = datas['브랜드'][i]
    a2 = datas['상품명'][i]
    a3 = datas['가격'][i]
    a4 = datas['별점개수'][i] if not None and not pd.isna(datas["별점개수"][i]) else "null"
    a5 = datas['좋아요개수'][i] if not None and not pd.isna(datas["좋아요개수"][i]) else "null"
    a6 = datas['시즌'][i] if not None and not pd.isna(datas["시즌"][i]) else ""
    a7 = datas['성별'][i] if not None and not pd.isna(datas["성별"][i]) else ""
    a8 = datas['별점'][i] if not None and not pd.isna(datas["별점"][i]) else "null"
    a9 = datas['이미지'][i] if not None and not pd.isna(datas["이미지"][i]) else ""

    print(a1, a2, a3, a4, a5, a6, a7, a8, a9)
    cur.execute(f"\
    insert into musinsa.musinsa(브랜드, 상품명, 가격, 별점개수, 좋아요개수, 시즌, 성별, 별점, 이미지)\
    value ('{a1}', '{a2}', {a3}, {a4}, {a5}, '{a6}', '{a7}', {a8}, '{a9}');")

드로우핏 오버사이즈 울 트렌치 코트 [BEIGE] 156700 7355.0 15866.0 2023 S/S 남 4.8 https://image.msscdn.net/images/goods_img/20180205/709189/709189_16768614323996_60.jpg
드로우핏 오버사이즈 울 트렌치 코트 [BLACK] 156700 4170.0 11668.0 2023 S/S 남 4.8 https://image.msscdn.net/images/goods_img/20180205/709188/709188_16768612544111_60.jpg
드로우핏 울 트렌치 싱글 발마칸 코트 [GREYISH BEIGE] 167200 117.0 1568.0 2023 S/S 남 4.9 https://image.msscdn.net/images/goods_img/20230118/3030654/3030654_16740931272826_60.jpg
굿라이프웍스 오버사이즈 트렌치코트 블랙 119800 344.0 3810.0 2021 F/W 남 4.8 https://image.msscdn.net/images/goods_img/20210826/2092842/2092842_4_60.jpg, https://image.msscdn.net/images/prd_img/20210826/2092842/detail_2092842_6_60.jpg, https://image.msscdn.net/images/prd_img/20210826/2092842/detail_2092842_7_60.jpg
플레어업 오버사이즈 트렌치코트 (FL-012) 156700 2573.0 8048.0 2023 ALL 남여 4.8 https://image.msscdn.net/images/goods_img/20171130/680961/680961_22_60.jpg, https://image.msscdn.net/images/prd_img/20171130/680961/detail_680961_37_60.jpg, https://image.mssc